<a href="https://colab.research.google.com/github/ElizavetaNosova/CompLing_homeworks/blob/master/Embeddings_ipynbgsHW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
! pip install pymorphy2[fast]
! pip install gensim
! pip install sklearn

     |████████████████████████████████| 51kB 1.9MB/s 
     |████████████████████████████████| 7.1MB 8.4MB/s 
     |████████████████████████████████| 256kB 50.6MB/s 
  Created wheel for DAWG: filename=DAWG-0.7.8-cp36-cp36m-linux_x86_64.whl size=774827 sha256=fde1449e74d3c20b4113dbc015fb3e5face09c48291adff005f5129fb4b46b84
  Stored in directory: /root/.cache/pip/wheels/d4/88/d0/4e4abc83eb8f59a71e8dbd8ba99fd5615a3af1fac1ef7f8825
Successfully built DAWG


In [0]:
import pandas as pd
from lxml import html
import numpy as np
from tqdm import tqdm
from matplotlib import pyplot as plt
from sklearn.decomposition import TruncatedSVD, NMF, PCA
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_distances
from sklearn.ensemble import RandomForestClassifier
import gensim
import numpy as np
from sklearn.cluster import MiniBatchKMeans
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from collections import Counter,defaultdict
from string import punctuation
import os
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
from pymorphy2 import MorphAnalyzer
%matplotlib inline

morph = MorphAnalyzer()
punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian'))

def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0].normal_form for word in words if word and word not in stops]

    return ' '.join(words)

def tokenize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]

    return ' '.join(words)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


1) Векторизуйте тексты с помощью Word2vec модели, обученной самостоятельно, и с помощью модели, взятой с rusvectores (любой). Обучите 2 модели по определению перефразирования на получившихся векторах и проверьте, что работает лучше. 
Word2Vec нужно обучить на отдельном корпусе (не на парафразах). Можно взять данные из семинара или любые другие. 
ВАЖНО: Оценивать модели нужно с помощью кросс-валидации! Метрика - f1.

In [0]:
from google.colab import files

uploaded = files.upload()

Saving lenta1000.csv to lenta1000.csv


In [0]:
import pandas as pd
lenta = pd.read_csv('lenta1000.csv')

Предполагаю, что на тысяче текстов качество будет так себе, но большой архив упорно не хотел загружаться на колаб. Надеюсь, что структура новостей в ленте (заголовок - полноцнное предложение, у которого есть синонимы в тексте) поможет выявить сколько-то синонимов

In [0]:
lenta

,Unnamed: 0,url,title,text,topic,tags
0,0,https://lenta.ru/news/2018/12/14/cancer/,Названы регионы России с самой высокой смертно...,Вице-премьер по социальным вопросам Татьяна Го...,Россия,Общество
1,1,https://lenta.ru/news/2018/12/15/doping/,Австрия не представила доказательств вины росс...,Австрийские правоохранительные органы не предс...,Спорт,Зимние виды
2,2,https://lenta.ru/news/2018/12/15/disneyland/,Обнаружено самое счастливое место на планете,Сотрудники социальной сети Instagram проанализ...,Путешествия,Мир
3,3,https://lenta.ru/news/2018/12/15/usa25/,В США раскрыли сумму расходов на расследование...,С начала расследования российского вмешательст...,Мир,Политика
4,4,https://lenta.ru/news/2018/12/15/integrity/,Хакеры рассказали о планах Великобритании зами...,Хакерская группировка Anonymous опубликовала н...,Мир,Общество
...,...,...,...,...,...,...
995,995,https://lenta.ru/news/2018/12/08/skater/,Американский фигурист «воткнул» партнершу в ле...,Американский фигурист Тимоти Ледук уронил парт...,Спорт,Зимние виды
996,996,https://lenta.ru/news/2018/12/08/vred/,В новой нефтяной сделке нашли вред для России,Новое соглашение членов Организации стран-эксп...,Экономика,Госэкономика
997,997,https://lenta.ru/news/2018/12/08/9m729/,Предсказано будущее ракеты 9М729 после ультима...,Ответом России на ультиматум США по Договору о...,Наука и техника,Оружие
998,998,https://lenta.ru/news/2018/12/08/paris/,В охваченный протестами Париж вошла бронетехника,"В Париж, где продолжаются антиправительственны...",Мир,Общество


In [0]:
lenta['title_and_text'] = lenta.title + ' ' + lenta.text

In [0]:
lenta

,Unnamed: 0,url,title,text,topic,tags,title_and_text
0,0,https://lenta.ru/news/2018/12/14/cancer/,Названы регионы России с самой высокой смертно...,Вице-премьер по социальным вопросам Татьяна Го...,Россия,Общество,Названы регионы России с самой высокой смертно...
1,1,https://lenta.ru/news/2018/12/15/doping/,Австрия не представила доказательств вины росс...,Австрийские правоохранительные органы не предс...,Спорт,Зимние виды,Австрия не представила доказательств вины росс...
2,2,https://lenta.ru/news/2018/12/15/disneyland/,Обнаружено самое счастливое место на планете,Сотрудники социальной сети Instagram проанализ...,Путешествия,Мир,Обнаружено самое счастливое место на планете С...
3,3,https://lenta.ru/news/2018/12/15/usa25/,В США раскрыли сумму расходов на расследование...,С начала расследования российского вмешательст...,Мир,Политика,В США раскрыли сумму расходов на расследование...
4,4,https://lenta.ru/news/2018/12/15/integrity/,Хакеры рассказали о планах Великобритании зами...,Хакерская группировка Anonymous опубликовала н...,Мир,Общество,Хакеры рассказали о планах Великобритании зами...
...,...,...,...,...,...,...,...
995,995,https://lenta.ru/news/2018/12/08/skater/,Американский фигурист «воткнул» партнершу в ле...,Американский фигурист Тимоти Ледук уронил парт...,Спорт,Зимние виды,Американский фигурист «воткнул» партнершу в ле...
996,996,https://lenta.ru/news/2018/12/08/vred/,В новой нефтяной сделке нашли вред для России,Новое соглашение членов Организации стран-эксп...,Экономика,Госэкономика,В новой нефтяной сделке нашли вред для России ...
997,997,https://lenta.ru/news/2018/12/08/9m729/,Предсказано будущее ракеты 9М729 после ультима...,Ответом России на ультиматум США по Договору о...,Наука и техника,Оружие,Предсказано будущее ракеты 9М729 после ультима...
998,998,https://lenta.ru/news/2018/12/08/paris/,В охваченный протестами Париж вошла бронетехника,"В Париж, где продолжаются антиправительственны...",Мир,Общество,В охваченный протестами Париж вошла бронетехни...


In [0]:
texts = list(lenta.title_and_text)

In [0]:
data_norm = [normalize(text) for text in texts]
data_norm = [text for text in data_norm if text]

In [0]:
data_norm[:5]

['назвать регион россия самый высокий смертность рак вице-премьер социальный вопрос татьяна голиков рассказать какой регион россия зафиксировать наиболее высокий смертность рак сообщать риа новость слово голиков чаща онкологический заболевание становиться причина смерть псковский тверская тульский орловский область также севастополь вице-премьер напомнить главное фактор смертность россия рак болезнь система кровообращение начало год стать известно смертность онкологический заболевание среди россиянин снизиться впервые год данные росстат 2017 год рак умереть 289 тысяча человек это 3,5 процент маленький год ранее',
 'австрия представить доказательство вина российский биатлонист австрийский правоохранительный орган представить доказательство нарушение российский биатлонист антидопинговый правило сообщить посол россия вена дмитрий любинский итог встреча уполномоченный адвокат дипмиссия представитель прокуратура страна передавать тасс действовать презумпция невиновность какой-либо ограничен

In [0]:
w2v = gensim.models.Word2Vec([text.split() for text in data_norm], size=50, sg=1)

In [0]:
uploaded = files.upload()

Saving paraphrases.xml to paraphrases.xml


In [0]:
uploaded = files.upload()

Saving paraphrases_no_head.xml to paraphrases_no_head.xml


In [0]:
tree = ET.ElementTree(file='paraphrases.xml')
root = tree.getroot()
for child_of_root in root:
    print(child_of_root.tag, child_of_root.attrib)


ParseError: ignored

Колаб ругается на плохо оформленный файл. Видимо, придётся извлекать текст регулярками. Почему подошёл индекс скобочной группы 2, а не 1, я не знаю

In [0]:
import re
paraphrases_df = pd.DataFrame({'text1': [], 'text2': [], 'class': [], 'jaccard': []})
with open('paraphrases.xml', 'r', encoding='utf=8') as f:
   text = f.read()
text = re.sub('Ё', 'Е', text) #затем будем использовать Ё как разделитель
text = re.sub('</?paraphrase>', 'Ё', text)
#print(text[:500])
paraphrases = re.findall('Ё[^Ё]+Ё', text)
for paraphrase_description in paraphrases:
    text1 = re.search('(<value name="text_1">)([^<]+)', paraphrase_description)
    text1 = normalize(text1.group(2))
    text2 = re.search('(<value name="text_2">)([^<]+)', paraphrase_description)
    text2 = normalize(text2.group(2))
    textclass = re.search('(<value name="class">)([^<]+)', paraphrase_description).group(2)
    jaccard = re.search('(<value name="jaccard">)([^<]+)', paraphrase_description).group(2)
    jaccard = float(re.sub(' ', '', jaccard))
    paraphrases_df.loc[len(paraphrases_df)+1] = [text1, text2, textclass, jaccard]

In [0]:
paraphrases_df

,text1,text2,class,jaccard
1,полицейский разрешить стрелять поражение гражд...,полиция мочь разрешить стрелять хулиган травма...,0,0.650000
2,право полицейский проникновение жилища решить ...,правило внесудебный проникновение полицейский ...,0,0.500000
3,президент египет ввести чрезвычайный положение...,власть египет угрожать ввести страна чрезвычай...,0,0.611429
4,вернуться сирия россиянин волновать вопрос тру...,самолёт мчс вывезти россиянин разрушить сирия,-1,0.324037
5,москва сирия вернуться 2 самолёт мчс россиянин...,самолёт мчс вывезти россиянин разрушить сирия,0,0.606218
...,...,...,...,...
7223,путин освободить должность ряд генералов,путин снять должность 20 руководитель-силовик,0,0.462500
7224,облако москва день победа разогнать девять сам...,путеводитель день победа провести 9 май москва,-1,0.457143
7225,люблян отпраздновать день победа вместе москва,москва ограничить движение связь днём победа,-1,0.584237
7226,девять самолёт ввс разогнать облако москва ден...,москва ограничить движение связь днём победа,-1,0.461880


In [0]:
paraphrases_df.to_csv('paraphrases_lemmas_no_tags.csv')

In [0]:
files.download('paraphrases_lemmas_no_tags.csv')

In [0]:
def text2vector(text):
  text = normalize(text)
  text = text.split()
  textvectors = []
  for word in text:
      try:
          textvectors.append(w2v[word])
      except KeyError:
          pass
  if textvectors:
      return [np.sum(textvectors, axis=0), True]
  else:
      return [np.zeros(50), False] #теоретически может случайно получиться сумма ноль, поэтому маркер вычисления вектора вынесла в отдельную переменную

1

In [0]:
first_texts = list(paraphrases_df.text1)
second_texts = list(paraphrases_df.text2)
textclass = list(paraphrases_df['class'])
all_feaches = []
real_classes = []
for i in range(len(first_texts)):
    vector1 = text2vector(first_texts[i])
    vector2 = text2vector(second_texts[i])
    if vector1[1] and vector2[1]:
        all_feaches.append(np.concatenate([vector1[0], vector2[0]]))
        

In [0]:
my_vectors_data

,vectors,class
0,"[-1.3110082, -0.24897516, -0.34025034, -1.9920...",0
1,"[-1.1590409, -0.09042022, 0.0381108, -1.717871...",0
2,"[-0.9237567, 0.8953589, 0.12389532, -2.8526745...",0
3,"[-1.2208245, 0.32234126, 0.23125248, -2.145326...",-1
4,"[-1.9691132, 0.23433006, -0.4408493, -3.371620...",0
...,...,...
7184,"[-0.7608899, 0.9300206, 0.33381748, -1.9772131...",0
7185,"[-1.0949445, 0.051020436, -0.53552425, -2.1961...",-1
7186,"[-0.9251123, 0.30018207, -0.3646065, -1.723035...",-1
7187,"[-1.0949445, 0.051020443, -0.53552425, -2.1961...",-1


In [0]:
len(my_vectors_data.vectors[1])

100

In [0]:
feaches = list(my_vectors_data.vectors)

NameError: ignored

In [0]:
def make_df_from_vectors(vectors, len_vector=100):
  dict_for_df = {}
  for i in range(len_vector):
    dict_for_df[i] = []
  my_vector_feaches = pd.DataFrame(dict_for_df)
  for vector in vectors:
      my_vector_feaches.loc[len(my_vector_feaches)] = vector
  return my_vector_feaches

In [0]:
feaches_df.to_csv('my_existed_vectors.csv')

In [0]:
from google.colab import files

In [0]:
files.download('my_existed_vectors.csv')

In [0]:
my_vectors_data.to_csv('my_existed_data.csv')

In [0]:
files.download('my_existed_data.csv')

In [0]:
X = feaches_df
y = my_vectors_data['class']

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [0]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression().fit(X_train, y_train)
predicted = clf.predict(X_test)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [0]:
from sklearn.metrics import f1_score
print(f1_score(y_test, predicted, average='micro'))
print(f1_score(y_test, predicted, average='macro'))

0.43882091212458285
0.3308714620119157


Не похоже на что-то вразумительное (впрочем, не удивительно при маленьком корпусе w2v).
Сделаем кросс-валидацию (кросс-валидация для всех пунктов задания - в конце тетрадки)

На примере полученных векторов посмотрим, как считать косинусное расстояние

In [0]:
import sklearn
def get_cos_distance(text1, text2):
    text1 = normalize(text1)
    text2 = normalize(text2)
    text1 = text1.split()
    text2 = text2.split()
    text1vectors = []
    text2vectors = []
    text1representation = np.zeros(50)
    text2representation = np.zeros(50)
    not_empty1 = False
    not_empty2 = False
    text2vectors = []
    for word in text1:
        try:
            text1vectors.append(w2v[word])
            not_empty1 = True
        except KeyError:
            pass
    for word in text2:
        try:
            text2vectors.append(w2v[word])
            not_empty2 = True
        except KeyError:
            pass
  #print(text1vectors)
  #print(text2vectors)
    if not_empty1 and not_empty2:
        text1vectors = np.array(text1vectors)
        text12ectors = np.array(text2vectors)
        text1representation = np.sum(text1vectors, axis = 0)
        text2representation = np.sum(text2vectors, axis = 0)
        #print(text1representation)
        cos =  sklearn.metrics.pairwise.cosine_distances([text1representation, text2representation]
        return cos
    else:
        return None

In [0]:
dummy_example = get_cos_distance('Путин и кошка посетили США, Россию и зоопарк', 'В России запретили заводить кошек и попугаев')
print(dummy_example)
print(sum(dummy_example[0]))


[[0.         0.03768897]
 [0.03768897 0.        ]]
0.0376889705657959


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  app.launch_new_instance()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [0]:
import sklearn
def get_cos_distance(text1, text2):
    text1 = normalize(text1)
    text2 = normalize(text2)
    text1 = text1.split()
    text2 = text2.split()
    text1vectors = []
    text2vectors = []
    text1representation = np.zeros(50)
    text2representation = np.zeros(50)
    not_empty1 = False
    not_empty2 = False
    text2vectors = []
    for word in text1:
        try:
            text1vectors.append(w2v[word])
            not_empty1 = True
        except KeyError:
            pass
    for word in text2:
        try:
            text2vectors.append(w2v[word])
            not_empty2 = True
        except KeyError:
            pass
    if not_empty1 and not_empty2:
        text1vectors = np.array(text1vectors)
        text12ectors = np.array(text2vectors)
        text1representation = np.sum(text1vectors, axis = 0)
        text2representation = np.sum(text2vectors, axis = 0)
        cos =  sklearn.metrics.pairwise.cosine_distances([text1representation, text2representation])
        return sum(cos[0])
    else:
        return None

Какой-то странный формат результата хочется верить, что это результат сравнения векторов дважды с соседом и дважды с самими собой (где нули). Поэтому во втором варианте функции буду возвращать сумму первого элемента cos

In [0]:
data_for_regression = []
first_texts = list(paraphrases_df.text1)
second_texts = list(paraphrases_df.text2)
class_data = list(paraphrases_df['class'])
print()
for i in range(len(first_texts)):
    cos = get_cos_distance(first_texts[i], second_texts[i])
    if cos:
      data_for_regression.append([cos, class_data[i]])

In [0]:
data_for_regression

In [0]:
cos = [i[0] for i in data_for_regression]
text_class = [i[1] for i in data_for_regression]
my_vectors_data = pd.DataFrame({'cos': cos, 'text_class':text_class})

In [0]:
my_vectors_data

,cos,text_class
0,0.010105,0
1,0.040688,0
2,0.020097,0
3,0.020606,-1
4,0.020729,0
...,...,...
7031,0.054010,0
7032,0.034439,-1
7033,0.029383,-1
7034,0.052708,-1


In [0]:
X = my_vectors_data['cos'].to_numpy()
y = my_vectors_data['text_class'].to_numpy()

In [0]:
type(y)

numpy.ndarray

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [0]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression().fit(X_train.reshape(-1, 1), y_train)
predicted = clf.predict(X_test.reshape(-1, 1))

In [0]:
from sklearn.metrics import f1_score

In [0]:
f1_score(predicted, y_test, average = 'micro')

0.46844798180784536

In [0]:
f1_score(predicted, y_test, average = 'macro')

0.34022222479114644

Результаты ужасные - впрочем, я и не рассчитывала получить ничего хорошего на 1000 текстов ленты.

Воспользуемся моделью rusvectores



In [0]:
!wget http://vectors.nlpl.eu/repository/11/180.zip

--2020-02-19 07:36:17--  http://vectors.nlpl.eu/repository/11/180.zip
Resolving vectors.nlpl.eu (vectors.nlpl.eu)... 129.240.189.225
Connecting to vectors.nlpl.eu (vectors.nlpl.eu)|129.240.189.225|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 484452285 (462M) [application/zip]
Saving to: ‘180.zip’

180.zip             100%[===================>] 462.01M  21.5MB/s    in 27s     

2020-02-19 07:36:45 (16.9 MB/s) - ‘180.zip’ saved [484452285/484452285]



In [0]:
import zipfile
#model_url = 'http://vectors.nlpl.eu/repository/11/180.zip'
#m = wget.download(model_url)
#model_file = model_url.split('/')[-1]
with zipfile.ZipFile('180.zip', 'r') as archive:
    stream = archive.open('model.bin')
    model = gensim.models.KeyedVectors.load_word2vec_format(stream, binary=True)


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
#у меня очень медленно работает mystem 
from google.colab import files
uploaded = files.upload()

Saving data_paraphraser_norm.csv to data_paraphraser_norm.csv


In [0]:
paraphraser_norm = pd.read_csv('data_paraphraser_norm.csv')

In [0]:
paraphraser_norm

,label,text_1,text_2,text_1_norm,text_2_norm
0,0,Полицейским разрешат стрелять на поражение по ...,Полиции могут разрешить стрелять по хулиганам ...,полицейский_NOUN разрешать_VERB стрелять_VERB ...,полиция_NOUN мочь_VERB разрешать_VERB стрелять...
1,0,Право полицейских на проникновение в жилище ре...,Правила внесудебного проникновения полицейских...,право_ADV полицейский_NOUN на_ADP проникновени...,правило_NOUN внесудебный_ADJ проникновение_NOU...
2,0,Президент Египта ввел чрезвычайное положение в...,Власти Египта угрожают ввести в стране чрезвыч...,президент_NOUN египет_NOUN вводить_VERB чрезвы...,власть_NOUN египет_NOUN угрожать_VERB вводить_...
3,-1,Вернувшихся из Сирии россиян волнует вопрос тр...,Самолеты МЧС вывезут россиян из разрушенной Си...,вернуться_VERB из_ADP сирия_NOUN россиянин_NOU...,самолет_NOUN мчс_NOUN вывозить_VERB россиянин_...
4,0,В Москву из Сирии вернулись 2 самолета МЧС с р...,Самолеты МЧС вывезут россиян из разрушенной Си...,в_ADP москва_NOUN из_ADP сирия_NOUN вернуться_...,самолет_NOUN мчс_NOUN вывозить_VERB россиянин_...
...,...,...,...,...,...
7222,0,Путин освободил от должности ряд генералов,Путин снял с должностей более 20 руководителей...,путин_NOUN освобождать_VERB от_ADP должность_N...,путин_NOUN снимать_VERB с_ADP должность_NOUN м...
7223,-1,Облака над Москвой в День Победы разгонят девя...,Путеводитель по Дню Победы: как провести 9 мая...,облако_NOUN над_ADP москва_NOUN в_ADP день_NOU...,путеводитель_NOUN по_ADP день_NOUN победа_NOUN...
7224,-1,Любляна отпразднует День Победы вместе с Москвой,В Москве ограничат движение в связи с Днем Победы,любляна_NOUN отпраздновать_VERB день_NOUN побе...,в_ADP москва_NOUN ограничивать_VERB движение_N...
7225,-1,Девять самолетов ВВС разгонят облака над Москв...,В Москве ограничат движение в связи с Днем Победы,девять_NUM самолет_NOUN ввс_NOUN разгонять_VER...,в_ADP москва_NOUN ограничивать_VERB движение_N...


In [0]:
def text2vector_rusvectores(text):
    text = text.split()
    vectors = []
    for word in text:
        try:
            vectors.append(w2v[word])
        except KeyError:
            pass
    if vectors:
         return np.sum(vectors, axis=0)
    else:
         return np.zeros(300)

In [0]:
w2v = model
vectors = []
labels = []
texts1 = list(paraphraser_norm['text_1_norm'])
texts2 = list(paraphraser_norm['text_2_norm'])
for text1, text2 in zip(texts1, texts2):
    vector1 = text2vector_rusvectores(text1)
    vector2 = text2vector_rusvectores(text2)
    vector = np.concatenate([vector1, vector2])
    vectors.append(vector)
        


In [0]:
dict_for_vectors = {}
for i in range(600):
    dict_for_vectors[i] = []
rusvectore_data = pd.DataFrame(dict_for_vectors)
for i in range(len(vectors)):
    rusvectore_data.loc[len(rusvectore_data)] = vectors[i]
rusvectore_data_with_class = rusvectore_data
rusvectore_data_with_class['class'] = paraphraser_norm.label

In [0]:
rusvectore_data_with_class.to_csv('rusvectore_data_with_class.csv')

In [0]:
files.download('rusvectore_data_with_class.csv')

In [0]:
X = rusvectore_data
y = paraphraser_norm.label

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [0]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression().fit(X_train, y_train)
predicted = clf.predict(X_test)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [0]:
from sklearn.metrics import f1_score
print(f1_score(y_test, predicted, average='micro'))
print(f1_score(y_test, predicted, average='macro'))

0.945213060320974
0.9458796135516906


С помощью модели, обученной на корпусе большого объема, получилось  намного более высокое качество работы

ЧАСТЬ 2

2) Преобразуйте тексты в векторы в каждой паре 5 методами  - SVD, NMF, Word2Vec (свой и  русвекторовский), Fastext. У вас должно получиться 5 пар векторов для каждой строчки в датасете. Между векторами каждой пары вычислите косинусную близость (получится 5 чисел для каждой пары)

Для этой задачи нецелесообразно пропускать предложения, которым модель не нашла векторное представление, поэтому использовать колонки датасетов из предыдущей части задания не получится.

SVD

In [0]:


def get_cos_distance(vector1, vector2):
    cos = sklearn.metrics.pairwise.cosine_distances([vector1, vector2])
    return sum(cos[0])
    
          

In [0]:
tfidf = TfidfVectorizer(min_df=3, max_df=0.4, max_features=1000)

In [0]:
tfidfmodel = tfidf.fit(data_norm)

In [0]:
svd = TruncatedSVD(200)

X_text_1 = svd.fit_transform(tfidfmodel.transform(paraphrases_df['text1']))
X_text_2 = svd.fit_transform(tfidfmodel.transform(paraphrases_df['text2']))

In [0]:
import sklearn

In [0]:
texts1 = list(paraphrases_df.text1)
texts2 = list(paraphrases_df.text2)
text_class = list(paraphrases_df['class'])

In [0]:
count_vectorizer_cos = [get_cos_distance(X_text_1[i], X_text_2[i]) for i in range(len(X_text_1))]

Мой w2v

In [0]:
def get_cos_distance_my_w2v(text1, text2):
    text1 = normalize(text1)
    text2 = normalize(text2)
    text1 = text1.split()
    text2 = text2.split()
    text1vectors = []
    text2vectors = []
    not_empty1 = False
    not_empty2 = False
    text2vectors = []
    for word in text1:
        try:
            text1vectors.append(w2v[word])
            not_empty1 = True
        except KeyError:
            pass
    for word in text2:
        try:
            text2vectors.append(w2v[word])
            not_empty2 = True
        except KeyError:
            pass
    if not_empty1 and not_empty2:
        text1vectors = np.array(text1vectors)
        text12ectors = np.array(text2vectors)
        text1representation = np.sum(text1vectors, axis = 0)
        text2representation = np.sum(text2vectors, axis = 0)
        cos =  sklearn.metrics.pairwise.cosine_distances([text1representation, text2representation])
        return sum(cos[0])
    else:
        return 0.5

In [0]:
my_w2v_cos = [get_cos_distance_my_w2v(texts1[i], texts2[i]) for i in range(len(texts1))]

italicized text

In [0]:
##NMF
cv = CountVectorizer(min_df=3, max_df=0.4, max_features=1000)
X = cv.fit_transform(data_norm)

In [0]:
nmf = NMF(50)
nmf.fit(X)

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
    n_components=50, random_state=None, shuffle=False, solver='cd', tol=0.0001,
    verbose=0)

In [0]:
X_text_1_nmf = nmf.transform(tfidf.transform(paraphrases_df['text1']))
X_text_2_nmf = nmf.transform(tfidf.transform(paraphrases_df['text2']))

In [0]:
cosNMF = [get_cos_distance(X_text_1_nmf[i], X_text_2_nmf[i]) for i in range(len(X_text_1_nmf))]

Rusvectors

In [0]:
def get_cos_distance_w2v(text1, text2):
    text1 = text1.split()
    text2 = text2.split()
    text1vectors = []
    text2vectors = []
    not_empty1 = False
    not_empty2 = False
    text2vectors = []
    for word in text1:
        try:
            text1vectors.append(w2v[word])
            not_empty1 = True
        except KeyError:
            pass
    for word in text2:
        try:
            text2vectors.append(w2v[word])
            not_empty2 = True
        except KeyError:
            pass
    if not_empty1 and not_empty2:
        text1vectors = np.array(text1vectors)
        text12ectors = np.array(text2vectors)
        text1representation = np.sum(text1vectors, axis = 0)
        text2representation = np.sum(text2vectors, axis = 0)
        cos =  sklearn.metrics.pairwise.cosine_distances([text1representation, text2representation])
        return sum(cos[0])
    else:
        return 0.5

In [0]:
texts1 = list(paraphraser_norm.text_1_norm)
texts2 = list(paraphraser_norm.text_2_norm)
w2v=model
rusvector_cos = [get_cos_distance_w2v(texts1[i], texts2[i]) for i in range(len(texts1))]

Fasttext

In [0]:
fast_text = gensim.models.FastText([text.split() for text in data_norm], size=50, 
                                   min_n=4, max_n=8)

In [0]:
def getvector_fasttext(text):
    text = text.split()
    vectors = []
    for word in text:
        try:
            vectors.append(fast_text[word])
        except KeyError:
            pass
    if vectors:
        return np.sum(np.array(vectors), axis=0)
    else:
        return np.zeros(50)

In [0]:
fasttextvectors1 = [getvector_fasttext(texts1[i]) for i in range(len(texts1))]
fasttextvectors2 = [getvector_fasttext(texts2[i]) for i in range(len(texts2))]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [0]:
fasttext_cos = [get_cos_distance(fasttextvectors1[i], fasttextvectors2[i]) for i in range(len(fasttextvectors1))]

In [0]:
all_cos = pd.DataFrame({'rusvector': rusvector_cos,'fasttext': fasttext_cos, 'my_w2v': my_w2v_cos, 'cv': count_vectorizer_cos,  'NMF': cosNMF, 'text_class': text_class})

In [0]:
all_cos.to_csv('all_cos.csv')

In [0]:
files.download('all_cos.csv')

In [0]:
all_cos = pd.read_csv('all_cos.csv')

In [0]:
X = all_cos.drop('text_class', axis=1)
y = all_cos.text_class

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [0]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression().fit(X_train, y_train)
predicted = clf.predict(X_test)

In [0]:
print(f1_score(y_test, predicted, average='micro'))
print(f1_score(y_test, predicted, average='macro'))

0.5589374654122855
0.4769802906544473


C пятью типами косинусных расстояний работает хуже, чем только с русвекторс без вычисления косинусных расстояний

Сдулаем кросс-валидацию для всех трёх задач с использованием уже полученных данных

In [0]:
##w2v - моя модель
from google.colab import files
uploaded = files.upload()

Saving my_existed_data.csv to my_existed_data.csv


In [0]:
X = pd.read_csv('my_existed_vectors.csv', index_col=0)

In [0]:
X

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
0,-1.311008,-0.248975,-0.340250,-1.992067,-0.139105,-0.032127,-0.076155,-0.239314,0.491036,-0.140604,-0.302940,-0.036335,-0.347080,0.907033,-0.049920,-0.439422,0.040510,-0.457198,-1.265820,-1.702954,0.494942,0.678699,-0.423003,-2.271011,0.003091,-0.712701,0.098896,0.851776,1.039470,0.516789,0.877060,-0.074973,-0.408402,-0.071862,-0.251796,-1.198694,-0.309328,-0.236817,-2.006337,-0.173055,...,-0.348343,0.102520,-0.215911,0.660795,-0.010149,-0.417014,-0.151916,-0.329753,-1.092657,-1.558959,0.354194,0.354065,-0.152676,-1.972251,-0.070930,-0.568488,0.170950,0.707940,0.899469,0.479140,0.431702,-0.244024,-0.373593,0.041137,-0.223628,-0.916934,-0.238360,-0.185444,-1.384313,-0.009521,1.264448,-0.611470,0.106642,0.440201,2.565839,1.100877,0.672132,-0.581036,0.079438,-1.044594
1,-1.159041,-0.090420,0.038111,-1.717872,0.037172,0.107198,0.021541,-0.321306,0.409480,-0.275182,-0.016309,0.278352,-0.163627,0.820600,0.090212,-0.394626,-0.125541,-0.340265,-1.170603,-1.353118,0.249228,0.171856,-0.312435,-1.882237,0.352919,-0.394006,0.223465,0.765786,1.304244,0.475647,0.337786,-0.296242,-0.372045,-0.104755,-0.254590,-1.031674,-0.342362,-0.415843,-1.667593,-0.103916,...,-0.516263,-0.040716,-0.143097,0.532014,0.063996,-0.140910,-0.256549,-0.191705,-0.642446,-1.053517,0.394413,0.328395,-0.057132,-1.400808,0.082773,-0.351152,0.291511,0.309202,0.811218,0.361983,0.039432,-0.088947,-0.463851,-0.056303,-0.196005,-0.639110,-0.147295,-0.332103,-1.372345,0.015282,1.128340,-0.440991,0.129393,0.333615,1.958473,0.773750,0.559938,-0.450290,0.021824,-0.975151
2,-0.923757,0.895359,0.123895,-2.852675,0.404712,0.118081,0.094719,-1.940471,0.722107,-0.920715,1.399936,0.659026,-0.210487,1.923994,-0.954894,0.602918,0.585354,-0.796176,-1.654751,-1.299015,0.929309,-0.138684,-2.510516,-1.784721,1.308928,0.098066,0.763046,1.568040,1.832126,-0.751285,0.752463,0.049950,-0.268294,-0.276239,0.455596,-1.474053,0.517645,-0.607124,-2.559069,-0.125256,...,0.863260,0.043602,0.048752,2.166984,-0.800136,-0.148340,0.681162,-0.341793,-2.491975,-0.837232,1.174698,0.131322,-3.150895,-2.237692,2.561689,-0.005797,1.053265,2.048362,2.157361,-0.447366,0.874536,-0.114913,-0.553319,-0.314024,0.746305,-1.566579,-0.076725,-0.531591,-2.761251,0.052932,2.890112,-0.889786,1.006964,-0.371682,5.000759,2.263218,0.775530,0.598671,-0.598262,-2.223687
3,-1.220824,0.322341,0.231252,-2.145326,0.045807,-0.305271,0.274307,-0.698308,0.702603,-0.261407,0.513962,0.367395,-0.375328,1.056875,0.089443,-0.348706,0.192221,-0.388241,-1.367554,-1.620470,0.396031,0.065632,-0.870272,-1.992593,0.366420,-0.548009,-0.098619,1.137796,1.305375,0.107057,0.865487,-0.391812,-0.138522,-0.074607,0.097740,-1.130048,0.067654,-0.433988,-1.922678,-0.283224,...,0.025805,0.231074,-0.399917,0.860742,-0.051716,-0.584805,0.429397,-0.164500,-1.536881,-1.469716,0.480075,0.575745,-0.923857,-2.289916,0.301340,-0.776435,0.036652,1.119174,1.072621,0.052044,0.981223,-0.211071,-0.329867,-0.052104,0.062554,-0.966862,-0.155674,-0.165618,-1.853164,-0.181584,1.576429,-0.585031,0.519560,0.303875,2.875142,1.415654,0.993269,-0.376441,-0.075422,-1.401406
4,-1.969113,0.234330,-0.440849,-3.371621,0.080148,0.388621,-0.697308,-1.600268,1.085970,-0.313957,-0.157998,-0.054772,-0.815249,1.580745,-0.338655,-0.632400,0.817385,-0.274310,-2.137359,-1.895230,0.900337,1.378170,-1.275428,-3.598218,0.747741,-1.095244,0.467014,1.722107,1.749590,-0.053342,1.605013,-0.369478,-0.939280,-0.000579,0.227157,-1.326869,-0.021677,-0.119795,-3.706947,0.145125,...,0.025805,0.231074,-0.399917,0.860742,-0.051716,-0.584805,0.429397,-0.164500,-1.536881,-1.469716,0.480075,0.575745,-0.923857,-2.289916,0.301340,-0.776435,0.036652,1.119174,1.072621,0.052044,0.981223,-0.211071,-0.329867,-0.052104,0.062554,-0.966862,-0.155674,-0.165618,-1.853164,-0.181584,1.57

In [0]:
y = pd.read_csv('my_existed_data.csv')['class']

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [0]:
n_fold = 10
folds = KFold(n_splits=n_fold, shuffle=True, random_state=0)
stratified_folds = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=0)
repeated_folds = RepeatedStratifiedKFold(n_splits=n_fold, n_repeats=20, random_state=0)

In [0]:
def cv_train_model(X, y, X_test, y_test, folds=folds, plot_feature_importance=False, model=None):
    prediction = np.zeros(len(X_test))
    valid_scores = []
    feature_importance = pd.DataFrame()
    for fold_n, (train_index, valid_index) in enumerate(folds.split(X, y)):
        X_train, X_valid = X[train_index], X[valid_index]
        y_train, y_valid = y[train_index], y[valid_index]
          

        model.fit(X_train, y_train)      
        y_pred_test = model.predict(X_test)
        y_pred_valid = model.predict(X_valid)
        valid_score = f1_score(y_valid, y_pred_valid, average='macro')
        
        valid_scores.append(valid_score)
        prediction += y_pred_test  
        
    prediction /= (fold_n + 1)
    test_score = f1_score(y_test, prediction, average='macro')
    print('Cross Validation mean score: {0:.4f}, std: {1:.4f}.'.format(np.mean(valid_scores), np.std(valid_scores)))
    print("Score on test data: {0:.4f}".format(test_score))

In [0]:
model1 = sklearn.linear_model.LogisticRegression()
model2 = sklearn.linear_model.LogisticRegression(C=0.5)
model3 = sklearn.linear_model.LogisticRegression(C=2)

In [0]:
X_train

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
349,-1.144555,-0.180738,-0.268660,-2.566048,0.239301,0.118868,0.033191,-0.979342,1.082604,0.285701,-0.883526,-0.411753,0.051552,1.860374,0.135277,0.101448,-0.660326,-0.071150,-0.768138,-1.218594,0.985678,0.184919,-1.307512,-2.331560,1.327911,-0.469251,0.627817,0.762036,2.132146,0.697464,-0.037661,-0.489269,-1.273287,-0.272486,0.105939,-1.374295,-0.459895,-1.145211,-3.880970,-0.334371,...,-0.106154,-0.030037,-0.076714,0.135558,-0.022892,-0.027653,-0.023148,-0.087208,-0.233993,-0.418644,0.112655,0.173130,-0.080023,-0.456057,-0.023050,-0.147191,0.072668,0.143916,0.170367,0.081610,0.144568,0.053808,-0.069241,-0.020003,-0.061108,-0.236937,0.011350,-0.064710,-0.409159,0.013316,0.318123,-0.192023,0.010937,0.109890,0.630593,0.316636,0.201553,-0.152344,0.009660,-0.309653
452,-1.102084,1.018204,-0.246298,-1.471660,-0.107958,0.083591,0.259912,-0.877863,0.075351,-0.238104,0.477495,0.150056,-0.177788,0.488618,0.137809,0.415481,0.064643,-0.268972,-0.614848,-1.409200,0.197740,0.065551,-0.469780,-0.731847,0.538576,0.076147,0.491378,0.376700,0.682918,-0.139392,-0.057107,0.229056,0.015588,-0.290822,0.055491,-0.713975,0.587531,-0.631412,-1.422124,0.034587,...,0.273571,0.154681,-0.199068,0.417274,0.089451,0.227728,0.031628,-0.379332,-0.715389,-1.576482,0.169109,0.082349,-0.249299,-0.940819,0.217243,-0.086407,0.321407,0.355188,0.621477,0.097231,0.064112,0.205673,0.037087,-0.220904,-0.125092,-0.760831,0.449403,-0.535637,-1.263109,-0.011951,0.591511,-0.778017,0.321372,0.229841,2.283345,0.969092,0.188738,-0.446956,-0.269786,-1.076639
662,-1.147255,-0.089104,0.034505,-2.202101,-0.124775,-0.276407,-0.160643,-0.614947,0.851018,0.031052,0.137327,-0.009247,-0.492447,1.132982,-0.124578,-0.522341,0.362309,-0.353958,-1.464372,-1.501185,0.411067,0.577085,-0.725578,-2.236889,0.329789,-0.804006,-0.181487,1.260521,1.084073,0.276897,1.360959,-0.275105,-0.242335,0.027329,-0.114945,-1.143834,-0.218670,-0.107636,-2.243478,-0.183181,...,0.161495,-0.152588,-0.555188,1.256531,-0.237620,-0.481430,0.483060,-0.638902,-1.614513,-1.896666,0.578669,0.903972,-1.032915,-2.462727,0.078298,-1.031327,-0.329844,1.402067,0.861170,0.427934,1.927123,-0.047788,-0.147398,-0.000319,-0.104322,-1.492932,-0.124373,-0.076752,-2.499663,-0.287048,1.724922,-0.608680,0.327773,0.403088,3.191929,2.191225,1.002112,-0.509304,0.039833,-1.384262
172,-0.810159,-0.112330,-0.514646,-1.184005,-0.121589,-0.006765,-0.093587,-0.154536,0.072264,-0.147635,-0.211876,-0.045221,-0.235910,0.467631,-0.182507,-0.144889,-0.010513,-0.279875,-0.724444,-1.215461,0.363585,0.548946,-0.259959,-1.359539,-0.143244,-0.445327,0.127943,0.470025,0.476528,0.255464,0.605511,0.204001,-0.183144,-0.058748,-0.211628,-0.727101,-0.003155,-0.102646,-1.186134,-0.014283,...,-0.552027,-0.072107,-0.063127,0.462613,-0.377078,0.054003,-0.086344,-0.382862,-0.493716,-1.146756,0.610745,0.739140,-0.054400,-1.382437,-0.241103,-0.399143,0.439478,0.300165,0.474826,0.159964,0.576252,0.240867,-0.439512,0.021678,-0.136529,-0.658108,0.133634,-0.112442,-1.201622,0.200434,1.372716,-0.690241,-0.164011,0.060993,2.005353,1.119503,0.972258,-0.360762,0.422156,-1.071839
1405,-1.390972,0.343716,0.071179,-2.617934,-0.068685,-0.076096,0.088771,-0.955217,0.885980,-0.300754,0.417778,0.202701,-0.493528,1.275271,0.064648,-0.392855,0.294971,-0.313371,-1.643771,-1.745071,0.519154,0.430865,-1.017389,-2.266308,0.702524,-0.619495,0.173610,1.263540,1.408572,0.029532,0.990545,-0.240395,-0.335400,-0.163386,-0.037970,-1.203892,-0.092223,-0.377291,-2.445738,-0.219912,...,0.502197,0.240387,-0.405266,1.341871,0.163795,-0.306620,0.197980,-0.160533,-1.377116,-1.689829,0.543467,0.337276,-0.995367,-2.143998,0.996016,-0.424573,0.472627,1.229997,1.575285,-0.135116,0.645513,-0.375884,-0.463291,-0.314004,0.189981,-1.160682,-0.072535,-0.588261,-2.647754,-0.2

In [0]:
cv_train_model(X_train.to_numpy(), y_train.to_numpy(), X_test.to_numpy(), y_test.to_numpy(), model=model1)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


ValueError: ignored

In [0]:
sklearn.model_selection.cross_val_score(model2, X, y)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


array([0.42837274, 0.44506259, 0.46940195, 0.38734353, 0.37021573])

In [0]:
sklearn.model_selection.cross_val_score(model3, X, y)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


array([0.42837274, 0.42280946, 0.48191933, 0.38803894, 0.36812804])

In [0]:
from sklearn.metrics import roc_auc_score

In [0]:
X_train.to_numpy()

array([[ 3.4900000e+02, -1.1445554e+00, -1.8073829e-01, ...,
        -1.5234390e-01,  9.6601040e-03, -3.0965292e-01],
       [ 4.5200000e+02, -1.1020845e+00,  1.0182042e+00, ...,
        -4.4695553e-01, -2.6978582e-01, -1.0766385e+00],
       [ 6.6200000e+02, -1.1472551e+00, -8.9103766e-02, ...,
        -5.0930446e-01,  3.9833285e-02, -1.3842623e+00],
       ...,
       [ 5.2260000e+03, -1.0675592e+00, -6.5306960e-01, ...,
        -7.4084103e-01,  8.9524090e-02, -2.0903792e+00],
       [ 5.3900000e+03, -1.2593403e+00,  1.3289862e+00, ...,
        -1.8124712e-01, -9.3362670e-01, -3.7545419e+00],
       [ 8.6000000e+02, -9.3246780e-01,  2.2281292e-01, ...,
         1.9135714e-02, -1.9031273e-01, -1.2526808e+00]])

In [0]:
cv_train_model(X_train.to_numpy(), y_train.to_numpy(), X_test.to_numpy(), y_test.to_numpy(), model=model1)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


AxisError: ignored

In [0]:
##часть 2
X = all_cos.drop('text_class', axis=1)
y = all_cos.text_class

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [0]:
cv_train_model(X_train.to_numpy(), y_train.to_numpy(), X_test.to_numpy(), y_test.to_numpy(), model=model1)

AxisError: ignored